This code attempts to derive the maintenance cost from MMS data. It reads two input files: the MMS WO (from the PTO) and the list of maintenance tasks (from the OEM). After matching the WO to the respective maintenance tasks from the list, the maintenance cost is calculated and outputted.

In [1]:
"""
setting up the notebook
"""
#pip install pyspellchecker first to download dependencies
#nltk.download('stopwords') first to download dependencies
#nltk.download('punkt') first to download dependencies
#nltk.download('wordnet') first to download dependencies
import pandas as pd
pd.set_option('display.max_colwidth', None)
from spellchecker import SpellChecker 
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from fuzzywuzzy import fuzz

C:\Users\tan_n\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
"""
reading input files
"""
wo_df = pd.read_excel('EDD_WO_CM.xlsx') #read WOs from MMS for subsystem of interest
task_df = pd.read_excel('EDD_Task_CM.xlsx') #read list of maintenace tasks for subsystem of interest
task_df = task_df.fillna('')

In [3]:
"""
define functions for data cleaning
"""
def clean(x): #remove typos
    spell = SpellChecker(distance=1)
    correct = []
    y = x.translate(str.maketrans('','',string.punctuation))
    mispelled = y.split()
    for word in mispelled:
        correct.append(spell.correction(word))
    return ' '.join(correct)

stop = stopwords.words('english') 
wordnet_lemmatizer = WordNetLemmatizer()
    
def tokenize_lemmatize(y): #tokenize and lemmatize string
    tokenized = nltk.word_tokenize(y) 
    lemmatized = []
    for word in tokenized:
        if word not in stop: #remove stop words
            lemmatized.append(wordnet_lemmatizer.lemmatize(word, pos='v'))
    return str(lemmatized)

In [4]:
"""
preprocessing task_df
"""
task_df['Combined Info'] = task_df['L2 Item Description'] + ' ' + task_df['L3 Item Description'] + ' ' + task_df['L4 Item Description'] + ' ' + task_df['L5 Item Description'] + ' ' + task_df['L6 Item Description'] + ' ' + task_df['Task Description'] + ' ' + task_df['Task']
task_df = task_df.drop(['L2 Item Description', 'L3 Item Description', 'L4 Item Description', 'L5 Item Description', 'L6 Item Description', 'Task Description', 'Task'], axis=1)
task_df['Combined Info'] = task_df['Combined Info'].str.lower()
task_df['Combined Info'] = task_df['Combined Info'].apply(lambda x: clean(x))
task_df['Combined Info'] = task_df['Combined Info'].apply(lambda y: tokenize_lemmatize(y))
task_df.head()

,Type,Code,Labour Cost [SGD],Material Cost\n(incl. consumables) (for non Replacement Task) [SGD],Spare Part Cost\n(for Replacement Task) [SGD],Combined Info
0,CM,2758,10.000000,0,50,"['edd', 'emergency', 'detainment', 'door', 'doorleaf', 'seal', 'seal', 'wear', 'due', 'age']"
1,CM,2759,10.000000,0,50,"['edd', 'emergency', 'detainment', 'door', 'doorleaf', 'seal', 'seal', 'removedmissing']"
2,CM,2760,23.333333,0,300,"['edd', 'emergency', 'detainment', 'door', 'doorleaf', 'lock', 'bolt', 'switch', 'ddmcs1ddmcs2', 'switch', 'break']"
3,CM,2761,5.000000,0,0,"['edd', 'emergency', 'detainment', 'door', 'doorleaf', 'lock', 'bolt', 'switch', 'ddmcs1ddmcs2', 'switch', 'power', 'due', 'connection', 'loose']"
4,CM,2762,5.000000,0,0,"['edd', 'emergency', 'detainment', 'door', 'doorleaf', 'lock', 'bolt', 'switch', 'ddmcs1ddmcs2', 'wrong', 'contact', 'side', 'activate']"


In [5]:
"""
preprocessing wo_df
"""
wo_df['Combined Info'] = wo_df['WO Text'] + ' ' + wo_df['Operation Text']
wo_df = wo_df.drop(['WO Text', 'Operation Text'], axis=1)
wo_df['Combined Info'] = wo_df['Combined Info'].str.lower()
wo_df['Combined Info'] = wo_df['Combined Info'].apply(lambda x: clean(x))
wo_df['Combined Info'] = wo_df['Combined Info'].apply(lambda y: tokenize_lemmatize(y))
wo_df.head()

,Order Number,Order Type,Actual Finish Date Time,Functional Location,Category,Sub Category,Man Hour,Combined Info
0,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']"
1,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']"
2,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']"
3,1928838,PM01,02/05/2021 11:15:00,8443/RSC/XDOR,XDOR,Nil,0.166667,"['8443', 'small', 'force', 'need', 'push', 'detainment', 'door', 'ramp', 'deploy', 'check', 'find', 'detainment', 'door', 'deployment', 'normal', 'function', 'need', 'push', 'small', 'force']"
4,1928838,PM01,02/05/2021 11:15:00,8443/RSC/XDOR,XDOR,Nil,0.166667,"['8443', 'small', 'force', 'need', 'push', 'detainment', 'door', 'ramp', 'deploy', 'check', 'find', 'detainment', 'door', 'deployment', 'normal', 'function', 'need', 'push', 'small', 'force']"


In [6]:
"""
define function for matching WO to maintenance task
"""
def match_task(task, list_task):
    max_score = -1
    task_index = 0
    max_task_index = 0
    
    for i in list_task:   
        score = fuzz.ratio(task, i)
        if (score > max_score):
            max_score = score
            max_task_index = task_index
        task_index += 1
    
    return (max_score, max_task_index)

In [7]:
"""
perform matching
"""
matched_score = []
matched_task_code = []
labour_cost = 0
material_cost = 0

for j in range(len(wo_df.index)):
    match = match_task(wo_df.iloc[j,-1], list(task_df['Combined Info']))
    matched_score.append(match[0])
    matched_task_code.append(task_df.iloc[match[1],1])
    labour_cost += task_df.iloc[match[1],2]
    material_cost += task_df.iloc[match[1],3] + task_df.iloc[match[1],4]

wo_df['Matched Task'] = matched_task_code
wo_df['Matched Score'] = matched_score
wo_df.head() #wo_df.to_excel('result.xlsx') to inspect output

,Order Number,Order Type,Actual Finish Date Time,Functional Location,Category,Sub Category,Man Hour,Combined Info,Matched Task,Matched Score
0,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']",2763,45
1,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']",2763,45
2,1871708,PM01,02/04/2019 14:40:00,8451/RSC/XDOR,XDOR,Nil,0.083333,"['pa', '45', 'check', 'detrainmnet', 'door', 'exercise', 'deploy', 'detainment', 'door', 'cab', 'end', 'end', 'able', 'deploy', 'check', 'ramp', 'damageabnormalities', 'find', 'function', 'test', 'peck', 'orc', 'able', 'activate', 'comms', 'reset', 'check', 'es', 'cover', 'normal']",2763,45
3,1928838,PM01,02/05/2021 11:15:00,8443/RSC/XDOR,XDOR,Nil,0.166667,"['8443', 'small', 'force', 'need', 'push', 'detainment', 'door', 'ramp', 'deploy', 'check', 'find', 'detainment', 'door', 'deployment', 'normal', 'function', 'need', 'push', 'small', 'force']",2852,55
4,1928838,PM01,02/05/2021 11:15:00,8443/RSC/XDOR,XDOR,Nil,0.166667,"['8443', 'small', 'force', 'need', 'push', 'detainment', 'door', 'ramp', 'deploy', 'check', 'find', 'detainment', 'door', 'deployment', 'normal', 'function', 'need', 'push', 'small', 'force']",2852,55


In [8]:
"""
calculating cost 
"""
print('Labour cost: $', round(labour_cost, 2)) #OEM assumed wage rate: $20/hr
print('Material cost: $', round(material_cost, 2))

Labour cost: $ 993.33
Material cost: $ 8970
